In [2]:
import glob
import os
from collections import Counter
from itertools import combinations, product

import numpy as np
np.random.seed(18012023)

import pandas as pd
pd.set_option('display.max_colwidth', 0)

import seaborn as sb

from sklearn.metrics import pairwise_distances
import lxml.etree
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import matplotlib
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.family'] = 'Arial'
from IPython.display import display

In /Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases

In [3]:
fig_dir = '../figures'
if not os.path.isdir(fig_dir):
    os.mkdir(fig_dir)

#### Load metadata

In [4]:
meta_df = pd.read_excel('../data/metadata_corrected.xlsx')
meta_df = meta_df[meta_df['exclude'] != 'x']
meta_df.sample(10)

,id,title,author,date,provenance,date_range,genre,subgenre,exclude
266,van_ses_vaerwen,Van ses vaerwen ende twaelf outheyden,NaN,1410.0,cdrom-mnl,1400-1420,Epiek,Didactiek,NaN
63,florigout_fragm_l,Florigout,NaN,1387.5,cdrom-mnl,1375-1400,Epiek,Ridder,NaN
111,loyhier_en_malaert_fragm_a,Loyhier en Malaert,NaN,1375.0,cdrom-mnl,1350-1400,Epiek,Karel,NaN
86,historie_van_gaver_capeel,Historie van Gaver Capeel,NaN,1510.0,cdrom-mnl,1500-1520,Epiek,Ridder,NaN
168,renout_van_montalbaen_fragm_be,Renout van Montalbaen,NaN,1350.0,cdrom-mnl,1340-1360,Epiek,Karel,NaN
211,spiegel_der_sonden,Spiegel der sonden,NaN,1450.0,cdrom-mnl,1440-1460,Epiek,Didactiek,NaN
210,spiegel_der_jongers,Spiegel der jongers,Lambertus Goetman,1515.0,cdrom-mnl,1510-1520,Epiek,Didactiek,NaN
123,madelgijs_fragm_p,Madelgijs,NaN,1387.5,cdrom-mnl,1375-1400,Epiek,Karel,NaN
166,reinaerts_historie,Reinaerts historie,NaN,1470.0,cdrom-mnl,1460-1480,Epiek,Dier,NaN
239,suverlijc_boecxken,Suverlijc boecxken,NaN,1508.0,cdrom-mnl,1508-1508,Lyriek,NaN,NaN


In [5]:
def get_verse_groups(verses, size=2, intertexts=False):
    for i in range(len(verses) - (size - 1)):
        if not intertexts:
            yield ' / '.join(verses[i : i + size])
        else:
            its = Counter(verses[i : i + size])
            if None in its:
                yield None
            elif len(its) > 1:
                yield 'overlap'
            else:
                yield list(its.keys())[0]

In [6]:
def parse_xml(fn, rm_interpol=False):
    try:
        tree = lxml.etree.parse(fn)
    except OSError:
        print(f'- Could not load {fn}')
        return None
        
    if rm_interpol:
        for interpolation in tree.xpath("//interpolation"):
            interpolation.getparent().remove(interpolation)
        
    for line_node in tree.iterfind('//l'):
        try:
            intertext_id = line_node.attrib['intertext']
        except KeyError:
            intertext_id = None
        
        tokens_ = line_node.attrib['tokens'].split()
        lemmas_ = []
        
        lemma_tags = [l.text for l in line_node.iterfind('.//lemma')]
        pos_tags = [p.text for p in line_node.iterfind('.//pos')]
        
        for lemma, pos in zip(lemma_tags, pos_tags):
            for l, p in zip(lemma.split('+'), pos.split('+')):
                    if p == 'n(prop)':
                        lemmas_.append('n(prop)')
                    else:
                        lemmas_.append(l)
    
        yield tokens_, lemmas_, intertext_id

In [7]:
GROUP_SIZE = 2

titles, tokens, lemmas, intertexts = [], [], [], []

for title, group in tqdm(meta_df.groupby('title')):
    work_tokens, work_lemmas, work_intertexts = [], [], []
    
    for id_ in sorted(group['id']):
        for tok, lem, intertext_id in parse_xml(f'../data/xml/{id_}.xml'):
            work_tokens.append(tok)
            work_lemmas.append(lem)
            work_intertexts.append(intertext_id)
    
    verse_tokens = [' '.join(v) for v in work_tokens]
    verse_lemmas = [' '.join(v) for v in work_lemmas]

    verse_group_tokens = list(get_verse_groups(verse_tokens, size=GROUP_SIZE))
    verse_group_lemmas = list(get_verse_groups(verse_lemmas, size=GROUP_SIZE))
    verse_group_intertexts = list(get_verse_groups(work_intertexts, size=GROUP_SIZE, intertexts=True))

    tokens.extend(verse_group_tokens)
    lemmas.extend(verse_group_lemmas)
    intertexts.extend(verse_group_intertexts)
    titles.extend([title] * len(verse_group_lemmas))

100%|██████████| 205/205 [00:33<00:00,  6.09it/s]


In [8]:
df = pd.DataFrame(zip(titles, tokens, lemmas, intertexts), columns=('title', 'tokens', 'lemmas', 'intertext'))

In [10]:
def tokenizer(text):
    return text.replace(' / ', ' ').lower().strip().split()

def add_rhyme_column(df):
    rhyme_words = []
    for lemmas in df['lemmas']:
        rhymes = []
        for verse in lemmas.split(' / '):
            rhymes.append(verse.strip().split()[-1])
        rhyme_words.append(' '.join(rhymes))
    df['rhyme'] = rhyme_words
    return df

In [11]:
df = add_rhyme_column(df)
df.head()

,title,tokens,lemmas,intertext,rhyme
0,AB recht ende averecht,Aensiet dese vrouwen hoe si gaen / Besiet hoe haer tuten staen,aanzien deze vrouw hoe zij gaan / bezien hoe zij de staan,None,gaan staan
1,AB recht ende averecht,Besiet hoe haer tuten staen / Claer dat si hen blanketten,bezien hoe zij de staan / klaar dat zij zij n(prop),None,staan n(prop)
2,AB recht ende averecht,Claer dat si hen blanketten / Die cleeder soe lanc dat si hen letten,klaar dat zij zij n(prop) / de kleed zo lang dat zij zij letten,None,n(prop) letten
3,AB recht ende averecht,Die cleeder soe lanc dat si hen letten / Ende sleypen hen nae al op die eerde,de kleed zo lang dat zij zij letten / en slapen zij na al op de aarde,None,letten aarde
4,AB recht ende averecht,Ende sleypen hen nae al op die eerde / Fi diere vuylder hoverde,en slapen zij na al op de aarde / fi duur vouwer hovaardij,None,aarde hovaardij


In [12]:
optim_vs = 7000
optim_rw = 0.1693877551020408
optim_th = 0.42202047865566794

In [13]:
vec = TfidfVectorizer(max_features=optim_vs, min_df=2,
                      tokenizer=tokenizer, token_pattern=None).fit(df['lemmas'])

## Nearest neighbors

In [14]:
def nearest_verses(title1, title2, base_df, vectorizer, prop_filt=2, rhyme_weight=.15):
    A = base_df[base_df['title'] == title1]
    B = base_df[base_df['title'] == title2]
    
    AX = vec.transform(A['lemmas']) + rhyme_weight * vec.transform(A['rhyme'])
    BX = vec.transform(B['lemmas']) + rhyme_weight * vec.transform(B['rhyme'])
    
    nearest = []
    for i, ax in enumerate(AX):
        distances = pairwise_distances(ax, BX, metric='cosine').flatten()
        top_i = distances.argsort()[0]
        nearest.append((A.iloc[i]['tokens'], B.iloc[top_i]['tokens'],
                        A.iloc[i]['lemmas'], B.iloc[top_i]['lemmas'],
                        A.iloc[i]['title'], B.iloc[top_i]['title'],
                        A.iloc[i]['intertext'], B.iloc[top_i]['intertext'],
                        distances[top_i]))

    nearest = pd.DataFrame(nearest,
                           columns=['tokens1', 'tokens2', 'lemmas1', 'lemmas2',
                                    'title1', 'title2', 'intertext1', 'intertext2',
                                    'distance'])

    nearest = nearest[nearest['title1'] != nearest['title2']]
    nearest = nearest[
                      (nearest['lemmas1'].str.count('n\(prop\)') < prop_filt) & \
                      (nearest['lemmas1'].str.count('n\(prop\)') < prop_filt) 
                     ]
    return nearest

In [15]:
nearest_df = nearest_verses('Moriaen', 'Lantsloot van der Haghedochte',
#nearest_df = nearest_verses('Limborch', 'Roman der Lorreinen',
#nearest_df = nearest_verses('Rijmkroniek van Woeringen', 'Grimbergse oorlog',
#nearest_df = nearest_verses('Parthonopeus van Bloys', 'Floris ende Blancefloer',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
#nearest_df.head(50).to_excel('../figures/mor-ele-selection.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
1712,Des men drinken mochte ende eten / Ende alles leets daer bi vergeten,Dat men drinken mocht ende eten / Ende leets bi vergheten,dat men drinken mogen en eten / en al leed daar bij vergeten,dat men drinken mogen en eten / en leed bij vergeten,Moriaen,Lantsloot van der Haghedochte,lan-ele-62,lan-ele-62,0.028858
892,Die es der werelt toeverlaet / Die late mi noch mine mesdaet,Die es der warelt toeuerlaet / Die late mj mine mesdaet,die zijn de wereld toeverlaat / die laten ik nog mijn misdaad,die zijn de wereld toeverlaat / die laten ik mijn misdaad,Moriaen,Lantsloot van der Haghedochte,lan-ele-52,lan-ele-52,0.046534
1808,Ende drogen des over een / Die daer waren in den steen,Si hebben gedreghen ouer een / Die daer sijn in den steen,en dragen de over een / die daar zijn in de steen,zij hebben dragen over een / die daar zijn in de steen,Moriaen,Lantsloot van der Haghedochte,mor-ele-40,mor-ele-40,0.061468
864,Alse lange alsi mochte leven / Getrouwelike sonder begeven,Also langhe alsic sal leuen / Ghetrouwelike sonder begheuen,alzo lang als zij mogen leven / getrouwelijk zonder begeven,alzo lang als ik zullen leven / getrouwelijk zonder begeven,Moriaen,Lantsloot van der Haghedochte,None,None,0.068830
1154,Maer dat was pine jegen spoet / Al haddemen hem gegeven alt goet,Mar dat was pine ieghen spoet / Die hare ghegheuen hadde alt goet,maar dat zijn pijn jegen spoed / al hebben men hij geven al dat goed,maar dat zijn pijn jegen spoed / die zij geven hebben al dat goed,Moriaen,Lantsloot van der Haghedochte,mor-ele-49,mor-ele-49,0.070083
2342,Tes icker weder come toe / Ende ic sal u vort secgen hoe,Des icker weder come toe / Ende sal v segghen hoe,dat ik er weder komen te / en ik zullen gij voort zeggen hoe,dat ik er weder komen te / en zullen gij zeggen hoe,Moriaen,Lantsloot van der Haghedochte,None,None,0.087825
326,Nu ne licgic nembermee / Boven enen nacht of twee,Dat hine laghe nemmermee / Bouen ere nacht ofte twee,nu ne ik liggen nimmermee / boven een nacht of twee,dat hij hij liggen nimmermee / boven een nacht of twee,Moriaen,Lantsloot van der Haghedochte,lan-ele-38,lan-ele-38,0.094797
964,Ende vrachden hem om niemare / Ofter enech ridder leden ware,Ende si vragheden hem om niemare / Of daer enich ridder leden ware,en vragen hij om nieuwmaar / af enig ridder lijden zijn,en zij vragen hij om nieuwmaar / of daar enig ridder lijden zijn,Moriaen,Lantsloot van der Haghedochte,None,lan-ele-36,0.104914
1620,Ende alse Walewein hadde verwonnen / Ende hem God hadde geonnen,Doe lantsloot hadde verwonnen / Des hem god hadde gheonnen,en als n(prop) hebben verwinnen / en hij god hebben gene,toen n(prop) hebben verwinnen / dat hij god hebben gene,Moriaen,Lantsloot van der Haghedochte,None,None,0.112967
1979,Der doet hi hadde daer menegen vrient / Die sere clageden haren toren,Hi hadde nochtan meneghen vrient / Die sere clagheden haren toren,de dood hij hebben daar menig vriend / die zeer klagen haar toorn,hij hebben nochtan menig vriend / die zeer klagen haar toorn,Moriaen,Lantsloot van der Haghedochte,lan-ele-56,lan-ele-56,0.117554


## Limborch-Lorreinen

In [16]:
nearest_df = nearest_verses('Limborch', 'Roman der Lorreinen',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
6723,Smergens doe die dach ontspranc / Entie liwerke sanc,Smargens als die dach ontspranc / Ende die lewerke sanc,de morgen toen de dag ontspringen / de en leeuwerik zingen,de morgen als de dag ontspringen / en de leeuwerik zingen,Limborch,Roman der Lorreinen,None,None,0.031846
14997,Smargens doe de dach ontspranc / Entie liwerke sanc,Smargens als die dach ontspranc / Ende die lewerke sanc,de morgen toen de dag ontspringen / de en leeuwerik zingen,de morgen als de dag ontspringen / en de leeuwerik zingen,Limborch,Roman der Lorreinen,None,None,0.031846
3335,Smargens doe die dach ontspranc / Entie liwerke sanc,Smargens als die dach ontspranc / Ende die lewerke sanc,de morgen toen de dag ontspringen / de en leeuwerik zingen,de morgen als de dag ontspringen / en de leeuwerik zingen,Limborch,Roman der Lorreinen,None,None,0.031846
9024,Ende seide here die macht van gode / Moet u behouden in u ere,Die seide gi heren die macht van gode / Moet v behouden in v ere,en zeggen heer de macht van n(prop) / moeten gij behouden in gij een,die zeggen gij heer de macht van n(prop) / moeten gij behouden in gij een,Limborch,Roman der Lorreinen,None,None,0.034808
2346,Die den keyser goeden dach / Ontboet tierst dat sine sach,Ende teerst datsi den keyser sach / Ontboet si heme goeden dach,de de keizer goed dag / ontbieden eerst het dat zijn zien,en eerst het dat zij de keizer zien / ontbieden zijn hij goed dag,Limborch,Roman der Lorreinen,None,None,0.039585
17189,Ic ligghe hier al miin leven lanc / Of ic hebs minen wille,Jc en hebbe van hem minen wille / Jc ligge hier al mijn leuen lanc,ik liggen hier al mijn leven lang / of ik hebben mijn wil,ik en hebben van hij mijn wil / ik liggen hier al mijn leven lang,Limborch,Roman der Lorreinen,None,None,0.058729
4429,Nu es die ridder in sorgen groet / Ende in anxste van der doet,Die nu sijn in anxste groet / Ende in sorgen van der doet,nu zijn de ridder in zorg groot / en in angst van de dood,die nu zijn in angst groot / en in zorg van de dood,Limborch,Roman der Lorreinen,None,None,0.060310
14979,Seits hem allen groten danc / Daerna over niet lanc,Gierbert seits hare groten danc / Daerna ouer niet lanc,n(prop) hij al groot dank / daarna over niet lang,n(prop) n(prop) haar groot dank / daarna over niet lang,Limborch,Roman der Lorreinen,None,None,0.071684
15115,Nu laet ic van hem bliven / Ende willu vort bescriven,Nu latic hier van hem bliuen / Ende wille vort bescriuen,nu laten ik van hij blijven / en n(prop) voorts beschrijven,nu ik laten hier van zij blijven / en willen voorts beschrijven,Limborch,Roman der Lorreinen,None,None,0.111559
14175,Gheseit heeft behagedem wale / Hi seide bi Mamet u tale,Huge seide gi segt wale / Mi behaegt wel uwe tale,zeggen hebben behagen wel / hij zeggen bij n(prop) uw taal,n(prop) zeggen gij zeggen wel / ik behagen wel uw taal,Limborch,Roman der Lorreinen,None,None,0.112073


In [17]:
rhyme_weight = .17
threshold = .5 # generous threshold!
A = df[df['title'] == 'Limborch'].iloc[nearest_df.iloc[0].name]
AX = vec.transform([A['lemmas']]) + rhyme_weight * vec.transform([A['rhyme']])

for title2 in df['title'].unique():
    B = df[df['title'] == title2]
    BX = vec.transform(B['lemmas']) + rhyme_weight * vec.transform(B['rhyme'])
    D = pd.Series(pairwise_distances(BX, AX, metric='cosine').min(axis=1))
    if D.min() < threshold:
        print(title2)
        top_idxs = D.argsort()[:3]
        top_hits = B.iloc[top_idxs]
        display(top_hits)

Arturs doet


,title,tokens,lemmas,intertext,rhyme
26413,Arturs doet,Dattie vrowe mochte wel versien / Des margens alse die dach ontspranc,dat de vrouw mogen wel verzien / de morgen als de dag ontspringen,None,verzien ontspringen
26414,Arturs doet,Des margens alse die dach ontspranc / Lanceloet stont op ende maecte sinen ganc,de morgen als de dag ontspringen / n(prop) staan op en maken zijn gang,None,ontspringen gang
26367,Arturs doet,Des mergens eer die sonne ontspranc / Quamen si gereden dat was waer,de morgen eer de zon ontspringen / komen zij rijden dat zijn waar,None,ontspringen waar


Der naturen bloeme


,title,tokens,lemmas,intertext,rhyme
159711,Der naturen bloeme,ende groet den dach alsi ontsprinct / jndien tiden dat soe sinct,en groot de dag als zij ontspringen / in de tijd dat zij zingen,None,ontspringen zingen
159710,Der naturen bloeme,die soete sinct alsic merke / ende groet den dach alsi ontsprinct,die zoet zingen als ik merken / en groot de dag als zij ontspringen,None,merken ontspringen
161924,Der naturen bloeme,der lewerken enter calandren / ende oec someghen uoglen andren,de leeuwerik en de kalander / en ook sommig vogel ander,None,kalander ander


Gedichten (Willem van Hildegaersberch)


...

,title,tokens,lemmas,intertext,rhyme
589991,Roman der Lorreinen,Smargens als die dach ontspranc / Ende die lewerke sanc,de morgen als de dag ontspringen / en de leeuwerik zingen,None,ontspringen zingen
589992,Roman der Lorreinen,Ende die lewerke sanc / Na dat ons die ieeste segt,en de leeuwerik zingen / na dat wij de geeste zeggen,None,zingen zeggen
583684,Roman der Lorreinen,Smargens doe ontspranc die dach / Ende menne scone baren sach,de morgen toen ontspringen de dag / en men ne schoon baren zien,None,dag zien


Sinte Lutgard


,title,tokens,lemmas,intertext,rhyme
650947,Sinte Lutgard,Dat si onsprongen es eer dach / Ende alse was onsprongen doe,dat zij ontspringen zijn eer dag / en als zijn ontspringen toen,None,dag toen
650948,Sinte Lutgard,Ende alse was onsprongen doe / Dat wijf began si al itoe,en als zijn ontspringen toen / dat wijf beginnen zij al ietoe,None,toen ietoe
660724,Sinte Lutgard,Was al volcomen so onspranc / Ver ysaeel ende oc lutgart,zijn al volkomen zo ontspringen / vrouw n(prop) en ook n(prop),None,ontspringen n(prop)


Van ses vaerwen ende twaelf outheyden


,title,tokens,lemmas,intertext,rhyme
891290,Van ses vaerwen ende twaelf outheyden,Uter erden hier ontspringhen / Daer van die blide voghel singhen,de uit aarde hier ontspringen / daar van de blij vogel zingen,None,ontspringen zingen
891289,Van ses vaerwen ende twaelf outheyden,Acoleyen tidelosen / Uter erden hier ontspringhen,n(prop) n(prop) / de uit aarde hier ontspringen,None,n(prop) ontspringen
891309,Van ses vaerwen ende twaelf outheyden,Vroylijc leven ende singhen / Wie XXIIII jaer heeft,vrolijk leven en zingen / wij 24 jaar hebben,None,zingen hebben
